In [2]:
import pandas as pd
import logging
import ETL.extract_data as extract_data
import ETL.config as config
import ETL.utils as utils
import polars as pl
import importlib
importlib.reload(config)
importlib.reload(extract_data)

# Configure logging
logger = logging.getLogger(__name__)

In [15]:
df = pd.read_csv(config.FILTERED_TRANSACTIONS_FILE_LOCAL,low_memory=True)
df.head()

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
0,18881487,2017-01-01 00:01:00,301,3742,$7.05,Online Transaction,39021,ONLINE,NaN,NaN,4784,NaN
1,18881488,2017-01-01 00:01:00,1129,2677,$100.00,Chip Transaction,27092,San Marcos,TX,78666.0,4829,NaN
2,18881490,2017-01-01 00:04:00,316,5857,$130.29,Chip Transaction,6389,Houston,TX,77047.0,5300,NaN
3,18881491,2017-01-01 00:04:00,566,3439,$12.39,Online Transaction,18563,ONLINE,NaN,NaN,4121,NaN
4,18881492,2017-01-01 00:04:00,1357,3649,$20.67,Chip Transaction,61195,Tacoma,WA,98404.0,5541,NaN


In [16]:
df['surrogate_key'] = df.index

In [ ]:
df.sample(3)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors,surrogate_key
1060814,20188073,2017-10-04 16:03:00,565,1314,$9.38,Swipe Transaction,75506,Carrollton,TX,75006.0,5921,NaN,1060814
2255417,21660842,2018-08-12 19:52:00,461,3172,$25.42,Chip Transaction,19964,Canton,MI,48188.0,5311,NaN,2255417
2436206,21883991,2018-09-29 08:01:00,348,3073,$49.25,Chip Transaction,18215,Trenton,NJ,8628.0,5719,NaN,2436206


: 

In [11]:
subset_col = ['merchant_id','merchant_city','merchant_state', 'zip', 'mcc']
duplicated_col = df[df.duplicated(subset=subset_col,keep=False)]
duplicated_col.sample(7)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
8366577,17672203,2016-04-18 01:35:00,1077,214,$3.05,Chip Transaction,43293,Ukiah,CA,95482,5499,NaN
8699440,18080908,2016-07-14 06:31:00,1478,5994,$7.82,Chip Transaction,17493,Lincoln,CA,95648,5812,NaN
236643,7758134,2010-03-13 16:01:00,642,5934,$49.72,Online Transaction,88459,ONLINE,NaN,0,5311,NaN
7554531,16676026,2015-09-16 21:35:00,1842,3743,$16.49,Chip Transaction,94974,Albuquerque,NM,87121,5812,NaN
10523931,20325215,2017-11-02 17:27:00,1581,5840,$31.75,Chip Transaction,67570,Ballwin,MO,63011,5311,NaN
7980744,17198376,2016-01-06 12:45:00,1453,1117,$10.15,Chip Transaction,45926,Phil Campbell,AL,35581,5814,NaN
11024631,20941144,2018-03-13 12:32:00,1181,5842,$14.41,Chip Transaction,5720,Concord,NC,28027,5310,NaN


In [12]:
df.loc[df['merchant_id'] == 41184].sort_values(by=['merchant_id','merchant_city','merchant_state', 'zip', 'mcc'])

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
745404,8368078,2010-08-10 14:27:00,497,2829,$18.75,Swipe Transaction,41184,Akron,IA,51001,5310,NaN
1375609,9130059,2011-02-08 15:50:00,229,5942,$0.92,Swipe Transaction,41184,Akron,IA,51001,5310,NaN
4520185,12961837,2013-06-30 22:13:00,194,5587,$13.51,Swipe Transaction,41184,Akron,IA,51001,5310,NaN
8803764,18208901,2016-08-10 12:02:00,120,3806,$81.89,Chip Transaction,41184,Akron,IA,51001,5310,NaN
13045988,23440661,2019-08-24 15:30:00,229,2683,$0.85,Swipe Transaction,41184,Akron,IA,51001,5310,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13264521,23710868,2019-10-21 06:03:00,197,1302,$41.52,Swipe Transaction,41184,Wilmore,KY,40390,5310,NaN
13272041,23720021,2019-10-23 06:04:00,197,1302,$39.79,Swipe Transaction,41184,Wilmore,KY,40390,5310,NaN
13294849,23748194,2019-10-29 00:45:00,197,1302,$27.95,Swipe Transaction,41184,Wilmore,KY,40390,5310,NaN
13298620,23752869,2019-10-30 06:12:00,197,1302,$48.79,Swipe Transaction,41184,Wilmore,KY,40390,5310,NaN


In [5]:
df['zip'] = df['zip'].fillna(0).astype(str).str.replace(r'\.0$', '', regex=True).astype(int)   

In [7]:
df.loc[df['zip'] == 0]

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
7,7475335,2010-01-01 00:14:00,1684,2140,$26.46,Online Transaction,39021,ONLINE,NaN,0,4784,NaN
8,7475336,2010-01-01 00:21:00,335,5131,$261.58,Online Transaction,50292,ONLINE,NaN,0,7801,NaN
18,7475346,2010-01-01 00:34:00,394,4717,$26.04,Online Transaction,39021,ONLINE,NaN,0,4784,NaN
24,7475353,2010-01-01 00:43:00,301,3742,$10.17,Online Transaction,39021,ONLINE,NaN,0,4784,NaN
26,7475356,2010-01-01 00:45:00,566,3439,$16.86,Online Transaction,16798,ONLINE,NaN,0,4121,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13305879,23761832,2019-10-31 23:22:00,1556,2972,$17.65,Online Transaction,88459,ONLINE,NaN,0,5311,NaN
13305880,23761833,2019-10-31 23:22:00,1797,5660,$34.81,Online Transaction,15143,ONLINE,NaN,0,4784,NaN
13305888,23761843,2019-10-31 23:33:00,1069,5167,$59.71,Online Transaction,39021,ONLINE,NaN,0,4784,NaN
13305897,23761853,2019-10-31 23:39:00,1422,5696,$694.30,Online Transaction,70268,ONLINE,NaN,0,4722,NaN


In [1]:
df['zip'] = df['zip'].str.replace('.0','')
df.head()

NameError: name 'df' is not defined

In [4]:
polars_df = pl.read_csv(config.TRANSACTIONS_FILE_LOCAL).lazy().collect() 
polars_df.head()

id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
i64,str,i64,i64,str,str,i64,str,str,f64,i64,str
7475327,"""2010-01-01 00:01:00""",1556,2972,"""$-77.00""","""Swipe Transaction""",59935,"""Beulah""","""ND""",58523.0,5499,null
7475328,"""2010-01-01 00:02:00""",561,4575,"""$14.57""","""Swipe Transaction""",67570,"""Bettendorf""","""IA""",52722.0,5311,null
7475329,"""2010-01-01 00:02:00""",1129,102,"""$80.00""","""Swipe Transaction""",27092,"""Vista""","""CA""",92084.0,4829,null
7475331,"""2010-01-01 00:05:00""",430,2860,"""$200.00""","""Swipe Transaction""",27092,"""Crown Point""","""IN""",46307.0,4829,null
7475332,"""2010-01-01 00:06:00""",848,3915,"""$46.41""","""Swipe Transaction""",13051,"""Harwood""","""MD""",20776.0,5813,null


In [10]:
importlib.reload(config)
df_2018 = df.query('date >= "2017-01-01"')  
df_2018.to_csv(config.FILTERED_TRANSACTIONS_FILE_LOCAL, index=False)

In [12]:
df_2018.info(show_counts=True)
print({df_2018['date'].min()})

<class 'pandas.core.frame.DataFrame'>
Index: 3954066 entries, 9351849 to 13305914
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   id              3954066 non-null  int64  
 1   date            3954066 non-null  object 
 2   client_id       3954066 non-null  int64  
 3   card_id         3954066 non-null  int64  
 4   amount          3954066 non-null  object 
 5   use_chip        3954066 non-null  object 
 6   merchant_id     3954066 non-null  int64  
 7   merchant_city   3954066 non-null  object 
 8   merchant_state  3473293 non-null  object 
 9   zip             3444171 non-null  float64
 10  mcc             3954066 non-null  int64  
 11  errors          62903 non-null    object 
dtypes: float64(1), int64(5), object(6)
memory usage: 392.2+ MB
{'2017-01-01 00:01:00'}


In [25]:
dim_merchant = df_2018.loc[:,['merchant_id', 'merchant_city', 'merchant_state','mcc']].drop_duplicates()
dim_merchant.shape


(114128, 4)

In [39]:
dim_merchant.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114128 entries, 9351849 to 13305819
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   merchant_id     114128 non-null  int64 
 1   merchant_city   114128 non-null  object
 2   merchant_state  114128 non-null  object
 3   mcc             114128 non-null  int64 
 4   zip             0 non-null       object
dtypes: int64(2), object(3)
memory usage: 5.2+ MB


In [34]:
online = dim_merchant.loc[dim_merchant['merchant_state'].isna()]
online.shape

(147, 4)

In [37]:
dim_merchant.loc[(dim_merchant['merchant_state'].isna()) & (dim_merchant['merchant_city'] == 'ONLINE'), ['merchant_state','zip']] = 'ONLINE'  
dim_merchant

,merchant_id,merchant_city,merchant_state,mcc,zip
9351849,39021,ONLINE,ONLINE,4784,NaN
9351850,27092,San Marcos,TX,4829,NaN
9351851,6389,Houston,TX,5300,NaN
9351852,18563,ONLINE,ONLINE,4121,NaN
9351853,61195,Tacoma,WA,5541,NaN
...,...,...,...,...,...
13305644,7688,Olyphant,PA,5812,NaN
13305702,42245,Mesa,AZ,7538,NaN
13305737,52038,Gibsonia,PA,5813,NaN
13305772,87481,Cleveland,OH,8049,NaN


In [36]:
dim_merchant.loc[dim_merchant['merchant_state'] == 'ONLINE']

,merchant_id,merchant_city,merchant_state,mcc
9351849,39021,ONLINE,ONLINE,4784
9351852,18563,ONLINE,ONLINE,4121
9351857,73186,ONLINE,ONLINE,4814
9351868,85247,ONLINE,ONLINE,5815
9351875,726,ONLINE,ONLINE,4900
...,...,...,...,...
12028681,79927,ONLINE,ONLINE,5912
12029327,41184,ONLINE,ONLINE,5310
12029625,65881,ONLINE,ONLINE,5311
12029783,15426,ONLINE,ONLINE,3390


In [157]:
dim_merchant.to_csv(config.DIM_MERCHANT_FILE, index=False)
dim_merchant.shape

(197452, 4)

In [143]:
dim_merchant.head()

,merchant_id,merchant_city,merchant_state,mcc
0,59935,Beulah,ND,5499
1,67570,Bettendorf,IA,5311
2,27092,Vista,CA,4829
3,27092,Crown Point,IN,4829
4,13051,Harwood,MD,5813


In [109]:
dim_merchant.groupby(['merchant_state','merchant_city'])['merchant_id'].nunique().reset_index()

,merchant_state,merchant_city,merchant_id
0,AA,Dpo,1
1,AK,Anchor Point,1
2,AK,Anchorage,89
3,AK,Barrow,7
4,AK,Bethel,1
...,...,...,...
18633,WY,Worland,8
18634,WY,Wright,2
18635,Yemen,Sanaa,3
18636,Zambia,Lusaka,5


In [111]:
dim_merchant['merchant_id'].unique().shape

(74831,)

In [67]:
df.loc[df['merchant_id'] == 59935,:].sort_values(by='merchant_city', ascending=True)

,id,date,client_id,card_id,amount,use_chip,merchant_id,merchant_city,merchant_state,zip,mcc,errors
11452949,21470128,2018-07-03 15:44:00,1752,3880,$33.24,Chip Transaction,59935,Abbeville,LA,70510.0,5499,NaN
12992421,23374480,2019-08-10 15:18:00,1752,3880,$33.00,Chip Transaction,59935,Abbeville,LA,70510.0,5499,NaN
12992404,23374458,2019-08-10 15:14:00,1752,3880,$73.00,Chip Transaction,59935,Abbeville,LA,70510.0,5499,NaN
12992364,23374414,2019-08-10 15:06:00,1752,3880,$-73.00,Chip Transaction,59935,Abbeville,LA,70510.0,5499,Bad PIN
12453555,22707922,2019-03-22 15:39:00,1752,3880,$-94.00,Chip Transaction,59935,Abbeville,LA,70510.0,5499,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12145825,22326308,2018-12-31 22:12:00,194,4273,$-95.00,Chip Transaction,59935,Zanesville,OH,43701.0,5499,NaN
12145832,22326315,2018-12-31 22:15:00,194,4273,$26.09,Chip Transaction,59935,Zanesville,OH,43701.0,5499,NaN
12134693,22312572,2018-12-28 22:04:00,194,5587,$71.00,Swipe Transaction,59935,Zanesville,OH,43701.0,5499,NaN
12142447,22322114,2018-12-30 22:29:00,194,4273,$83.00,Chip Transaction,59935,Zanesville,OH,43701.0,5499,NaN


In [62]:
dim_merchant['merchant_id'].value_counts()
dim_merchant.loc[dim_merchant['merchant_id'] == 59935,:].sort_values(by='merchant_city', ascending=True)

,merchant_id,merchant_city,merchant_state
9373948,59935,Abbeville,LA
9373896,59935,Aberdeen,MD
9364780,59935,Abilene,TX
12008354,59935,Absecon,NJ
10710269,59935,Acapulco,Mexico
...,...,...,...
12401703,59935,Youngstown,OH
9892370,59935,Ypsilanti,MI
11819678,59935,Yuma,AZ
11352790,59935,Zanesville,OH


In [169]:
import pandas as pd
import json
import logging

logger = logging.getLogger(__name__)

def extract_json_mcc_data(json_path:str, data_name:str) -> pd.DataFrame:
    """
    Extracts data from a JSON file and returns it as a pandas DataFrame.
    
    Parameters:
    json_path (str): The path to the JSON file.
    data_name (str): The name of the data to be extracted.
    
    Returns:
    pd.DataFrame: DataFrame containing the extracted data.
    """
    try:
        
        with open(json_path, 'r') as f:
            mcc_dict = json.load(f)

        # Convert dictionary to DataFrame
        df = pd.DataFrame(list(mcc_dict.items()), columns=['mcc', 'description'])
        df['mcc'] = df['mcc'].astype(int)

        return df
    
        logger.info(f'✅ Extracted {data_name} data with {len(df)} records')
                
    except Exception as e:
        logger.error(f'❌Error extracting data from {data_name}: {e}')
        raise

test_mcc = extract_json_mcc_data(config.MCC_FILE_LOCAL, 'mcc_codes')
test_mcc

,mcc,description
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees
...,...,...
104,7549,Towing Services
105,5941,Sporting Goods Stores
106,5722,Household Appliance Stores
107,3009,Fabricated Structural Metal Products


In [176]:
def transform_dim_merchant(df: pd.DataFrame, data_name: str) -> pd.DataFrame:
    """
    Transforms the merchant data by merging with MCC codes and adding a merchant key.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing the merchant data.
    df2 (pd.DataFrame): DataFrame containing the MCC codes.
    
    Returns:
    pd.DataFrame: Transformed DataFrame with additional columns.
    """
    try:
        dim_merchant = df.copy()
        dim_merchant = df.loc[:,['merchant_id', 'merchant_city', 'merchant_state','mcc']].drop_duplicates()
        mcc_df = extract_json_mcc_data(config.MCC_FILE_LOCAL, 'MCC')
        dim_merchant_mcc = dim_merchant.merge(mcc_df, left_on='mcc', right_on='mcc', how='left').reset_index(drop=True)
        dim_merchant_mcc['merchant_key'] = dim_merchant_mcc.index + 1
        
        return dim_merchant_mcc
        logger.info(f'✅ Successfully transformed {data_name} with {len(dim_merchant_mcc)} records')
    except Exception as e:
        logger.error(f'❌Error transforming data: {e}')
        raise

 

In [177]:
merchant_cleaned = transform_dim_merchant(df, 'dim_merchant')
merchant_cleaned.head()

,merchant_id,merchant_city,merchant_state,mcc,description,merchant_key
0,59935,Beulah,ND,5499,Miscellaneous Food Stores,1
1,67570,Bettendorf,IA,5311,Department Stores,2
2,27092,Vista,CA,4829,Money Transfer,3
3,27092,Crown Point,IN,4829,Money Transfer,4
4,13051,Harwood,MD,5813,Drinking Places (Alcoholic Beverages),5


In [ ]:
       
# Load JSON file
with open(r'C:\Users\jimmy\Documents\Data Projects\Project\Financial Transactions Project\data\mcc_codes.json') as f:
    mcc_dict = json.load(f)

# Convert dictionary to DataFrame
mcc_df = pd.DataFrame(list(mcc_dict.items()), columns=['mcc_code', 'description'])
mcc_df['mcc_code'] = mcc_df['mcc_code'].astype(int)
mcc_df.rename(columns={'mcc_code': 'mcc'}, inplace=True)

mcc_df


In [ ]:
dim_merchant_mcc = dim_merchant.merge(mcc_df, left_on='mcc', right_on='mcc', how='left').reset_index(drop=True)



In [150]:
dim_merchant_mcc['merchant_key'] = dim_merchant_mcc.index + 1
dim_merchant_mcc.head()

,merchant_id,merchant_city,merchant_state,mcc,description,merchant_key
0,59935,Beulah,ND,5499,Miscellaneous Food Stores,1
1,67570,Bettendorf,IA,5311,Department Stores,2
2,27092,Vista,CA,4829,Money Transfer,3
3,27092,Crown Point,IN,4829,Money Transfer,4
4,13051,Harwood,MD,5813,Drinking Places (Alcoholic Beverages),5


In [129]:
distinct_mcc = dim_merchant.groupby('merchant_id')['mcc'].nunique().reset_index()
distinct_mcc.loc[distinct_mcc['mcc'] > 1,:]

,merchant_id,mcc
48,73,2
424,569,2
468,631,2
596,795,2
666,889,2
...,...,...
74052,99259,2
74218,99482,2
74385,99724,2
74616,100042,2


In [100]:
# Comprehensive summary of a column
def column_summary(df, column_name):
    print(f"Column: {column_name}")
    print(f"Total rows: {len(df)}")
    print(f"Unique values: {df[column_name].nunique()}")
    print(f"Unique values (including NaN): {df[column_name].nunique(dropna=False)}")
    print(f"Missing values: {df[column_name].isnull().sum()}")
    print(f"Data type: {df[column_name].dtype}")
    
column_summary(dim_merchant, 'merchant_id')

Column: merchant_id
Total rows: 197423
Unique values: 74831
Unique values (including NaN): 74831
Missing values: 0
Data type: int64


In [200]:
def extract_data(file_path: str, data_type: str, columns:list) -> pd.DataFrame:
    """
    Generic function to extract data from CSV files and return as DataFrame.
    Logs the number of records loaded or any errors encountered during the process.
    
    Args:
        file_path (str): Path to the CSV file to extract
        data_type (str): Name of the data type for logging purposes (e.g., 'cards', 'transactions', 'users')
        columns (list): optional, If you need to filter columns in the extracting
    Returns:
        pd.DataFrame: DataFrame containing the extracted data
        
    Raises:
        Exception: Re-raises any exception encountered during CSV reading
    """
    try:
        df = pd.read_csv(file_path,usecols=columns)
        logger.info(f"✅ Extracted {data_type} data.csv with {len(df)} records")
        return df
    except Exception as e:
        logger.error(f"❌ Error extracting {data_type}_data.csv: {e}")
        raise

columns1 = ['merchant_id', 'merchant_city', 'merchant_state', 'mcc']
merchant = extract_data(r"C:\Users\jimmy\Documents\Data Projects\Project\Financial Transactions Project\data\filtered_transactions_data.csv", 'MERCHANT',columns1)


In [201]:
merchant

,merchant_id,merchant_city,merchant_state,mcc
0,20561,Houston,TX,5912
1,58192,La Habra,CA,4900
2,39021,ONLINE,NaN,4784
3,42636,Hallandale,FL,7802
4,87819,Miami,FL,5812
...,...,...,...,...
2554753,86438,West Covina,CA,5499
2554754,39261,ONLINE,NaN,5815
2554755,2925,Allen,TX,4900
2554756,46284,Daly City,CA,5411
